In [1]:
"""
    Author: Iris Shakya
    course: Big Data Technology
    Project: 2, tf-idf w/ similarity search( mapreduce + pyspark)
    instructor: Prof. Lei Xei
    References: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.zipWithIndex.html
                https://towardsdatascience.com/tf-idf-calculation-using-map-reduce-algorithm-in-pyspark-e89b5758e64c
"""
import math


In [2]:
from pyspark import SparkConf, SparkContext
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
import gc
from pyspark.sql.functions import monotonically_increasing_id

sc = SparkContext(master = 'local')
spark = SparkSession.builder.appName('pySpark word-count').config('spark.some.config.option', 'some-value')             .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/17 14:09:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.sparkContext.textFile("/Users/iris/project2_demo.txt")

import regex as re
regex_st = r'\d{8}'

# doc = data.map(lambda x: x.split(regex_st))
doc = data.flatMap(lambda x: x.split())
doc1 = doc.map(lambda x: (x.split(regex_st)))

In [4]:
my_para = data.map(lambda x: tuple(x.split('\n')))

In [5]:
#split_list = my_para.map(lambda x: list(x[0].split()))
split_list = my_para.map(lambda x: tuple(x[0].split()))
#split_list2 = split_list.filter(lambda x: x[0] is not None)


doc_id = split_list.zipWithIndex().keyBy(lambda x: x[1])

# unpacking 2 tuple into 1 singular tuple to conpute
doc_id = doc_id.map(lambda x: tuple((x[0], *x[1])))

# re-packing teh tuple
doc_id = doc_id.map(lambda x: tuple((x[0], x[1])))
doc_id.take(3)


[(0,
  ('10022814',
   'growth',
   'inhibition',
   'dis_breast_cancer_dis',
   'cell',
   'grb2',
   'downregulation',
   'correlate',
   'inactivation',
   'mitogen-activated',
   'protein',
   'kinase',
   'gene_egfr+_gene',
   'erbb2',
   'cell',
   'increased',
   'dis_breast_cancer_dis',
   'growth',
   'associate',
   'increase',
   'expression',
   'gene_epidermal_growth_factor_gene',
   'receptor',
   'gene_egfr+_gene',
   'erbb2',
   'receptor',
   'tyrosine',
   'kinase',
   'rtks',
   'upon',
   'activation',
   'rtks',
   'transmit',
   'oncogenic',
   'signal',
   'binding',
   'growth',
   'factor',
   'receptor',
   'bind',
   'protein-2',
   'grb2',
   'turn',
   'bind',
   'sos',
   'activate',
   'ras-raf-mek-mitogen-activated',
   'protein',
   'map',
   'kinase',
   'pathway',
   'grb2',
   'important',
   'transformation',
   'fibroblast',
   'gene_egfr+_gene',
   'erbb2',
   'whether',
   'grb2',
   'important',
   'proliferation',
   'dis_breast_cancer_dis',
  

In [6]:
# separate into key-value pairs
# ('doc_id', 'term') --> (( 'doc_id', 'token'), 1 )
kv_mapper = doc_id.flatMap(lambda x: tuple( [ ((x[0], i),1) for i in x[1]]))

In [7]:
kv_mapper.take(5)

[((0, '10022814'), 1),
 ((0, 'growth'), 1),
 ((0, 'inhibition'), 1),
 ((0, 'dis_breast_cancer_dis'), 1),
 ((0, 'cell'), 1)]

In [8]:
# do the term-count
# (('doc_id', 'token'), 1) --> (('doc_id', 'token'), [1++])
reduce_1 = kv_mapper.reduceByKey(lambda x, y: x+y)
reduce_1.collect()

[((0, '10022814'), 1),
 ((0, 'growth'), 6),
 ((0, 'inhibition'), 3),
 ((0, 'dis_breast_cancer_dis'), 7),
 ((0, 'cell'), 6),
 ((0, 'grb2'), 7),
 ((0, 'downregulation'), 2),
 ((0, 'correlate'), 1),
 ((0, 'inactivation'), 2),
 ((0, 'mitogen-activated'), 1),
 ((0, 'protein'), 3),
 ((0, 'kinase'), 4),
 ((0, 'gene_egfr+_gene'), 5),
 ((0, 'erbb2'), 5),
 ((0, 'increased'), 1),
 ((0, 'associate'), 1),
 ((0, 'increase'), 1),
 ((0, 'expression'), 1),
 ((0, 'gene_epidermal_growth_factor_gene'), 1),
 ((0, 'receptor'), 3),
 ((0, 'tyrosine'), 1),
 ((0, 'rtks'), 4),
 ((0, 'upon'), 2),
 ((0, 'activation'), 2),
 ((0, 'transmit'), 1),
 ((0, 'oncogenic'), 1),
 ((0, 'signal'), 1),
 ((0, 'binding'), 1),
 ((0, 'factor'), 1),
 ((0, 'bind'), 2),
 ((0, 'protein-2'), 1),
 ((0, 'turn'), 1),
 ((0, 'sos'), 1),
 ((0, 'activate'), 1),
 ((0, 'ras-raf-mek-mitogen-activated'), 1),
 ((0, 'map'), 2),
 ((0, 'pathway'), 2),
 ((0, 'important'), 2),
 ((0, 'transformation'), 1),
 ((0, 'fibroblast'), 1),
 ((0, 'whether'), 1),
 

In [9]:
# mapping for IDF start
# (('doc_id', 'token'), tf) --> ('token', ('doc_id', tf))
shuffle = reduce_1.map(lambda x: (x[0][1], (x[0][0], x[1])))
shuffle.take(1)

[('10022814', (0, 1))]

In [10]:
# tokenise for idf
# (('doc_id', 'token'), tf) --> ('token', ('doc_id', tf, 1))
mapper_2 = reduce_1.map(lambda x: (x[0][1], (x[0][0], x[1],1)))
mapper_2.take(2)

[('10022814', (0, 1, 1)), ('growth', (0, 6, 1))]

In [11]:
# only take term and token
# ('token', ('doc_id', tf,1)) --> ('token', 1)
mapper_3 = mapper_2.map(lambda x: (x[0], x[1][2]))
mapper_3.take(3)

[('10022814', 1), ('growth', 1), ('inhibition', 1)]

In [12]:
doc_id.take(3)

[(0,
  ('10022814',
   'growth',
   'inhibition',
   'dis_breast_cancer_dis',
   'cell',
   'grb2',
   'downregulation',
   'correlate',
   'inactivation',
   'mitogen-activated',
   'protein',
   'kinase',
   'gene_egfr+_gene',
   'erbb2',
   'cell',
   'increased',
   'dis_breast_cancer_dis',
   'growth',
   'associate',
   'increase',
   'expression',
   'gene_epidermal_growth_factor_gene',
   'receptor',
   'gene_egfr+_gene',
   'erbb2',
   'receptor',
   'tyrosine',
   'kinase',
   'rtks',
   'upon',
   'activation',
   'rtks',
   'transmit',
   'oncogenic',
   'signal',
   'binding',
   'growth',
   'factor',
   'receptor',
   'bind',
   'protein-2',
   'grb2',
   'turn',
   'bind',
   'sos',
   'activate',
   'ras-raf-mek-mitogen-activated',
   'protein',
   'map',
   'kinase',
   'pathway',
   'grb2',
   'important',
   'transformation',
   'fibroblast',
   'gene_egfr+_gene',
   'erbb2',
   'whether',
   'grb2',
   'important',
   'proliferation',
   'dis_breast_cancer_dis',
  

In [13]:
# counting by token
# ('token', 1) --> ('token', [1++])
reducer_2 = mapper_3.reduceByKey(lambda x, y: x+y)
reducer_2.take(3)

[('10022814', 1), ('growth', 5731), ('inhibition', 2070)]

In [14]:
# convert into python list so we can pass it into a map transformation
doc_id1 = doc_id.map(list).collect()


In [15]:
idf = reducer_2.map(lambda x: (x[0], math.log(len(doc_id1)/x[1])))
idf.take(3)

[('10022814', 9.277438115508676),
 ('growth', 0.6237928009569395),
 ('inhibition', 1.6421342292492622)]

In [21]:
joint_my = shuffle.join(idf).take(5)
joint_my
type(joint_my)

list

In [23]:
# parallelize so we can convert list back into a RDD
rdd = sc.parallelize(joint_my)

In [25]:
# ('doc_id', 'everything_else') --> (('doc_id'), ('token', 'tf', 'idf', 'tf-idf'))
mapper_4 = rdd.map(lambda x: (x[1][0][0], ( x[0], x[1][0][1], x[1][1], \
    x[1][0][1] * x[1][1])) ).sortByKey()
mapper_4.take(4)

[(0, ('10022814', 1, 9.277438115508676, 9.277438115508676)),
 (0, ('growth', 6, 0.6237928009569395, 3.742756805741637)),
 (1, ('growth', 3, 0.6237928009569395, 1.8713784028708185)),
 (5, ('growth', 8, 0.6237928009569395, 4.990342407655516))]